In [1]:
import cupy as cp
import pygpe.spintwo as gpe
from helper import get_linear_interp, get_tf_density_3d
import matplotlib.pyplot as plt

In [2]:
NX = 128
NY = 128
NZ = 128
DX = 20 / NX
DY = 20 / NY
DZ = 20 / NZ

points = (NX, NY, NZ)
grid_spacings = (DX, DY, DZ)
grid = gpe.Grid(points, grid_spacings)

In [12]:
params = {
    "c0": 1.32e4,
    "c2": 146,
    "c4": -129,
    "p": 0.0,
    "q": 0.3 * (get_linear_interp(grid.z_mesh) - 0.5),
    "trap": 0.5 * (grid.x_mesh**2 + grid.y_mesh**2 + grid.z_mesh**2),
    "nt": 100,
    "dt": -1j * 1e-2,
    "t": 0,
}

In [13]:
eta = get_linear_interp(grid.z_mesh)
tf = get_tf_density_3d(params["c0"], params["c2"], grid.x_mesh, grid.y_mesh, grid.z_mesh, N=1)

psip2 = cp.sqrt(tf) * cp.sqrt((1 - eta**2) / 2)
psip1 = cp.random.normal(scale=1e-4, size=grid.shape)
psi0 = cp.sqrt(tf) * eta
psim1 = cp.random.normal(scale=1e-4, size=grid.shape)
psim2 = cp.sqrt(tf) * cp.sqrt((1 - eta**2) / 2)

psi = gpe.Wavefunction(grid)
psi.set_custom_components(psip2, psip1, psi0, psim1, psim2)
psi.fft()

In [14]:
for i in range(params["nt"]):
    gpe.step_wavefunction(psi, params)
psi.ifft()